# Scrape HN

This file scrapes this month's HN Jobs post using Selenium

In [ ]:
#| default_exp scraper

#| export
who_is_hiring = "https://news.ycombinator.com/item?id=40846428"


In [ ]:
%pip install -q selenium
%pip install -q nbdev
%pip install -q webdriver-manager
%pip install -q pillow

In [ ]:
#| export
from nbdev.showdoc import *

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [ ]:
#| export
def getPosts(postsHtml: list):
    posts = []

    for post in postsHtml:
        indent = post.find_element(By.CLASS_NAME, "ind").get_attribute("indent")

        # ignore reply comments: comments indented by 1+
        # later on it might be useful to attach these to the parent comment
        if (indent == "0"):
            hnuser = post.find_element(By.CLASS_NAME, "hnuser").text
            date = post.find_element(By.CLASS_NAME, "age").get_attribute("title")
            comment = post.find_element(By.CLASS_NAME, "comment").text

            post = {
                "hnuser": hnuser,
                "date": date,
                "comment": comment    
            }

            posts.append(post)

    return posts

In [ ]:
#| export

def scrape(url: str):
    url = url or who_is_hiring

    #create an instance of the Service object
    service = Service(executable_path=ChromeDriverManager().install())
    #start Chrome using the service keyword
    driver = webdriver.Chrome(service=service)
    driver.get(url)

    tbody = driver.find_element(By.CLASS_NAME, "comment-tree")
    postsHtml = tbody.find_elements(By.CLASS_NAME, "athing")

    return getPosts(postsHtml)

    driver.quit()

In [ ]:
#| export
import json

def savePosts(filename, posts):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(posts, f, ensure_ascii=False, indent=4)

In [ ]:
#| export
import json

def loadJSON(filename):
    posts = False
    try: 
        with open(filename, 'r', encoding='utf-8') as f:
            posts = json.load(f)
    finally:
        return posts

In [65]:
#| hide
import nbdev; nbdev.nbdev_export()
